# Dog breed classification challenge

## I have used three types of network definitions:

### 1. Pre-trained models:
###    Resnet50
###    InceptionNet
###    Xception
###    Vgg16
### 2. Transfer learning and Fine Tuning on Xception Net
### 3. Make your own ConvNet model from scratch

## Transfer learning helps when we want to make weights initialization from an already trained network.
## Fine Tuning helps to train specific layers of the networks.
## We should leverage these two concepts when we have a very large dataset and when we can find some network weights which have been previously trained on a similar dataset

## Setup the Data

In [1]:
from classifier.load_data import *
from classifier.models import Models
from classifier.train import *

print('TRAINING DATA PREP')
filename_list, labels_list, annotation_list = load_matfile('data/lists/train_list.mat')
data_frame = create_dataframe(filename_list[:200], labels_list[:200], annotation_list[:200], 'data/annotation/')
pickle_file(data_frame=data_frame, file_to_save='train_data.pickle')

x_train = to_numpy_array(data_frame[:200], image_shape=(224, 224), data_path='data/images/')
y_train = labels_to_logical(labels_list[:200])

print('TEST DATA PREP')
filename_list, labels_list, annotation_list = load_matfile('data/lists/test_list.mat')
data_frame = create_dataframe(filename_list[:200], labels_list[:200], annotation_list[:200], 'data/annotation/')
pickle_file(data_frame=data_frame, file_to_save='test_data.pickle')

x_test = to_numpy_array(data_frame[:200], image_shape=(224, 224), data_path='data/images/')
y_test = labels_to_logical(labels_list[:200])

#Load the dictionary mappings for the labels
with open(r"data/labels_dict.pickle", "rb") as input_file:
    dictionary_labels_classes = pickle.load(input_file)


Using TensorFlow backend.


TRAINING DATA PREP
Loading the mat files
0
100
Cropping, resizing and saving as a numpy array
0
100
Conversion to one-hot encoding
TEST DATA PREP
Loading the mat files
0
100
Cropping, resizing and saving as a numpy array
0
100
Conversion to one-hot encoding


## Train and Test XceptionNet for Transfer Learning and Fine Tuning

In [ ]:
print('COMPILE and TRAIN MODEL')
model = Models()
model = model.TransferFine(top_layers=True)
history, model = train(model, x_train, y_train, split=0.8, early_stopping=True, epochs=5)

print('Saving: Model Architecture and Weights')
model.save('save_architecture.h5')
model.save_weights('save_model_weights.h5')


## Predict on some test images

In [ ]:
print('PREDICTING CLASSES')
   
classes = []
class_dogs = []
test = x_test[:10]
for i in range(len(test)):
    pred = np.argmax(model.predict(np.expand_dims(resize(test[i], (299, 299)), axis=0)))+1
    classes.append(pred)
    class_dogs.append(dictionary_labels_classes[pred])

print('Predicted Labels')
print(classes)
print(class_dogs)

print('True Labels')
print(labels_list[:10])

## Let us see some curves and a test image

In [ ]:
print('Let us see some training and validation curves')

# list all data in history
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#plot a test image
io.imshow(x_test[0])
plt.show()
print('So, it was actually a', dictionary_labels_classes[labels_list[0]])

## Test an image on ResNet50 pre-trained network

In [5]:
from keras.applications.vgg16 import preprocess_input, decode_predictions

print('Load and Test Model')
model = Models.PreTrained()
model = model.resnet50()

test_image = np.expand_dims(resize(x_test[20], (224, 224)), axis=0)
preds_resnet = model.predict(test_image)
_, imagenet_class_name, prob = decode_predictions(preds_resnet, top=1)[0][0]

print('Predicted Label')
print(imagenet_class_name, prob)

print('True Label')
print(labels_list[20], dictionary_labels_classes[labels_list[20]])


Load and Test Model
Predicted Label
(u'nematode', 0.15411375)
True Label
(1, u'Chihuahua')


## Test an image on Inception_V3 pre-trained network

In [9]:
from keras.applications.vgg16 import preprocess_input, decode_predictions

print('Load and Test Model')
model = Models.PreTrained()
model = model.Inception_V3()

test_image = np.expand_dims(resize(x_test[20], (299, 299)), axis=0)
preds_inception = model.predict(test_image, batch_size=32)
_, imagenet_class_name, prob = decode_predictions(preds_inception, top=1)[0][0]
print(imagenet_class_name, prob)

print('Predicted Label')
print(imagenet_class_name, prob)

print('True Label')
print(labels_list[20], dictionary_labels_classes[labels_list[20]])

Load and Test Model
(u'Chihuahua', 0.91658884)
Predicted Label
(u'Chihuahua', 0.91658884)
True Label
(1, u'Chihuahua')


### Thanks and let me know if you have any questions!